In [1]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import re

import nltk
from nltk import word_tokenize
from nltk.corpus import wordnet # pos
from nltk.corpus import stopwords
from sklearn.feature_extraction import _stop_words as stop_words

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

from functions import *

In [2]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Dataset

In [3]:
# hand-labeled dataset. 
# Cleaned (not duplicates). 
# Resized to have the same size for all three classes
data = []
data_labels = []
with open("Data/neg_u_13k.csv", encoding="utf8") as f:
  for i in f: 
    #data.append(eval(i).decode())  # we don eval/decode because each line is a string of a binary string
    #data.append(i.replace("b'", '').replace('b"', '').replace('"\n', '').replace("'\n", ''))
    data.append(i)
    data_labels.append('neg')
with open("Data/pos_u_13k.csv", encoding="utf8") as f:
  for i in f: 
    data.append(i)
    data_labels.append('pos')
with open("Data/neu_u_13k.csv", encoding="utf8") as f:
  for i in f: 
    data.append(i)
    data_labels.append('neu')


import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', -1)

df = pd.DataFrame(zip(data, data_labels), columns=['tweet', 'sentiment'])
non_test, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(non_test, test_size=0.25, random_state=42)

In [4]:
df.iloc[:20]

,tweet,sentiment
0,b'- THEN let it fuck u in the ass'\n,neg
1,"b'!!!!!!!!!!!!!!!!!!!! @ms_fabdee: For the love of God, please smell nice.""""'\n",neg
2,"b'"" 6 children, 2 adults dead in Florida shooting: BELL, Fla. (AP) \xe2\x80\x94 A once-convicted felon killed six ... http://t.co/BZw4fdoaZz #science'\n",neg
3,"b'"" Hopes for the future..? Yes. However , getting there with you...is not much different than being alone..!""'\n",neg
4,"b'"" If you don\'t have anything nice to say, don\'t say anything at all.""'\n",neg
5,"b""- isn't there, or that his beard looks weird, or that this whole fucking situation is completely odd!""\n",neg
6,"b""- Like don't Nobody Want Me Frfr ,""\n",neg
7,"b'"" Love,you hurt my heart ""'\n",neg
8,"b'"" Mfs stay feeling some type about shit I post on MY twitter ..Like if it bothers you so bad pray to Jesus about it maybe he\'ll fix it \xf0\x9f\x98\xb9\xf0\x9f\x91\x8c""'\n",neg
9,"b'"" Mr Cereal lover, I wish your mother loved you like I would of that way you could of known how to love a woman "" \xf0\x9f\x91\x8f\xf0\x9f\x99\x8c'\n",neg


### Baselines

##### Random Selection Model
A random classification model would have around 33% accuracy as there are three classes and they are balanced.

##### NLTK API

In [5]:
import requests
def nltk_label(text):
  res = requests.post('http://text-processing.com/api/sentiment/', data={'text':text})
  sentiment = eval(res.text)['label']
  return sentiment if sentiment != 'neutral' else 'neu'

# correct_count = 0
# for index, (tweet, sentiment) in test.iterrows():
#   if index >= 1000:  # because of throttle
#     break
#   try:
#     if nltk_label(tweet) == sentiment:
#       correct_count += 1
#   except Exception as e:
#     print(e)
#     print(tweet)
#     break
  
    
# print(round( (correct_count/1000 * 100), 2))

##### Simple Rule-based

In [6]:
rule_pos = set()
rule_neg = set()
with open("Data/rule_neg.txt", encoding="utf8") as f:
  for i in f: 
    rule_neg.add(i.rstrip('\n').lower())
with open("Data/rule_pos.txt", encoding="utf8") as f:
  for i in f: 
    rule_pos.add(i.rstrip('\n').lower())

def label(string):
  pos_count, neg_count = 0, 0
  for i in string.split():
    if i.lower() in rule_pos:
      pos_count += 1
    if i.lower() in rule_neg:
      neg_count += 1
  if pos_count == neg_count:
    if pos_count == 0:
      return 'neu'
    else: 
      return 'neg'
  if pos_count > neg_count:
    return 'pos'
  return 'neg'

correct_count = 0
for index, (tweet, sentiment) in df.iterrows():
  if label(tweet) == sentiment:
    correct_count += 1
    
print(round( (correct_count/len(df) * 100), 2))

#TODO: Apply stemming (lover, loving, loved, etc all should be turned into love)

38.61


### Baseline Model

In [18]:
def run_model(data, data_labels, vectorizer_params={}, model_params={}, model=None, verbose=False):  # for Naive Bayes: model = MultinomialNB()
  
  vectorizer = CountVectorizer(**vectorizer_params)
  features = vectorizer.fit_transform(data)
  
  X_train, X_val, y_train, y_val  = train_test_split(features, data_labels, test_size=0.2, random_state=42)
  
  if not model:
    model = LogisticRegression(**model_params, solver='newton-cg')
  
  if verbose:
    #print(vectorizer)
    print('Number of Features: ', len(vectorizer.get_feature_names()))
    #print('model: ', model)
    
  model.fit(X=X_train, y=y_train)
  y_pred = model.predict(X_val)

  return round(accuracy_score(y_val, y_pred) * 100, 2)

In [25]:
print('Accuracy: {}%'.format(run_model(train['tweet'], train['sentiment'], verbose=True)))

Number of Features:  47531
Accuracy: 59.82%


##### Simple Cleanup

In [23]:
def cleanup(text):
  '''Remove non-informative words and characters (punctuation, extra space, ...)'''
  tokens = text.split()
  filtered_tokens = ' '.join([ i for i in tokens if not i.startswith('http') 
                                                #and not i.startswith('@') 
                                                #and len(i) > 1 
                            ])
  return filtered_tokens

train2 = train.copy()
train2['tweet'] = train['tweet'].apply(cleanup)
print('Accuracy: {}%'.format(run_model(train2['tweet'], train2['sentiment'], verbose=True)))

Number of Features:  42281
Accuracy: 60.01%


In [24]:
def cleanup(text):
  '''Remove non-informative words and characters (punctuation, extra space, ...)'''
  tokens = text.split()
  filtered_tokens = ' '.join([ i for i in tokens if not i.startswith('http') 
                                                and not i.startswith('@') 
                                                #and len(i) > 1 
                            ])
  return filtered_tokens

train2 = train.copy()
train2['tweet'] = train['tweet'].apply(cleanup)
print('Accuracy: {}%'.format(run_model(train2['tweet'], train2['sentiment'], verbose=True)))

Number of Features:  33821
Accuracy: 60.37%


In [22]:
def cleanup(text):
  '''Remove non-informative words and characters (punctuation, extra space, ...)'''
  tokens = text.split()
  filtered_tokens = ' '.join([ i for i in tokens if not i.startswith('http') 
                                                and not i.startswith('@') 
                                                and len(i) > 1 
                            ])
  return filtered_tokens

train2 = train.copy()
train2['tweet'] = train['tweet'].apply(cleanup)
print('Accuracy: {}%'.format(run_model(train2['tweet'], train2['sentiment'], verbose=True)))

Number of Features:  33821


60.37

In [62]:
tokens = get_tokens(non_test['tweet'])
len(tokens)

381327

In [56]:
a = [ i for i in Counter(tokens).most_common(50) if i not in custom_stopwords ]
print(a)

[('the', 7235), ('RT', 7168), ('to', 7080), ('you', 5355), ('and', 3950), ('in', 3375), ('is', 3260), ('of', 3252), ('my', 3073), ('for', 3051), ('me', 2299), ('on', 2264), ('it', 2067), ('that', 2000), ("I'm", 1881), ('be', 1833), ('so', 1785), ('this', 1618), ('with', 1584), ('your', 1509), ('have', 1488), ('like', 1448), ('at', 1409), ('just', 1267), ('are', 1261), ('but', 1229), ('not', 1202), ('love', 1157), ("don't", 1067), ('was', 1024), ('all', 984), ('get', 973), ('up', 907), ('do', 906), ('can', 898), ('when', 874), ('&amp;', 844), ('out', 838), ('from', 818), ('if', 818), ('know', 783), ('we', 779), ('about', 778), ('will', 760), ('what', 747), ('The', 742), ("it's", 716), ('as', 711), ('they', 708), ('people', 687)]


#### Adding StopWords

In [10]:
# sklearn stopwords (default)
print('sklearn stopwords', run_model(non_test['tweet'], non_test['sentiment'], vectorizer_params={'stop_words':'english'}, verbose=True))

# sklearn stopwords (default) cleaned
from sklearn.feature_extraction import _stop_words as stop_words
scikit_stopwords = stop_words.ENGLISH_STOP_WORDS
scikit_stopwords_cleaned = set(scikit_stopwords)
# removing the following words from the list of stopwords as they could be helpful in determining sentiment
for member in ['empty', 'less', 'too', 'alone', 'never', 'enough', 'can', 'everything', 'give', 'serious', 
               'will', 'always', 'couldnt', 'nobody', 'must', 'sincere', 'cant', 'down', 'cannot', 'cry', 
               'full', 'neither', 'nowhere', 'anything', 'nor', 'nothing', 'not', 'please', 
               'last', 'behind', 'out', 'every']:
  scikit_stopwords_cleaned.remove(member)
print('sklearn stopwords cleaned', run_model(non_test['tweet'], non_test['sentiment'], vectorizer_params={'stop_words':scikit_stopwords_cleaned}, verbose=True))

# nltk stopwords
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('english')
print('nltk stopwords', run_model(non_test['tweet'], non_test['sentiment'], vectorizer_params={'stop_words':nltk_stopwords}, verbose=True))

# nltk stopwords cleaned
nltk_stopwords_cleaned=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "between", "into", "through", "during", "before", "after", "to", "from", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "more", "most", "other", "some", "such", "nor", "only", "own", "same", "so", "than", "very", "s", "t", "just", "don", "should", "now"]
print('nltk stopwords cleaned', run_model(non_test['tweet'], non_test['sentiment'], vectorizer_params={'stop_words':nltk_stopwords_cleaned}, verbose=True))

# the following is taken from the 100 most common tokens in the data that seems to have no sentiment
custom_stopwords = {'I', 'the', 'RT', 'to', 'a', 'you', 'and', 'in', 'is', 'of', 'my', 'for', 'me', 'on', 'it'
  , 'that', "I'm", 'be', 'so', 'this', 'with', 'your', 'have', 'at', 'just', 'are', 'but', 'i' 
  , 'was', 'all', 'get', 'up', 'do', 'when', '&amp;', '-', 'from', 'if', 'know', 'we', 'about', 'what'
  , 'The', "it's", 'as', 'they', 'u', 'people', 'one', 'by', 'no', 'see', 'go', 'You', 'how'
  , 'or', 'an', 'got', 'who', 'he', 'more', 'day', 'make', 'time'
  , "you're", 'My', 'A', 'has', 'really', 'much', 'now', 'some', "It's", 'back', 'If'
  , 'would', 'still', 'going', 'his', 'been', 'new', 'only', 'her', 'even'
  , "I've", 'them', 'than', 'our', "that's", 'This', 'had', '.', '2', 'there', 'she'
  , 'then', 'last', 'over', 'life', 'say', 'come'
  , 'their', 'am', 'because',
}

print('custome stopwords', run_model(non_test['tweet'], non_test['sentiment'], vectorizer_params={'stop_words':custom_stopwords}, verbose=True))

Number of Features:  31092
sklearn stopwords 58.17
Number of Features:  31124
sklearn stopwords cleaned 58.5
Number of Features:  31246
nltk stopwords 57.85
Number of Features:  31276
nltk stopwords cleaned 58.28


C:\Users\vvaezian\AppData\Roaming\Python\Python39\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['amp', 're', 'rt', 've'] not in stop_words.
  warnings.warn(


Number of Features:  31304
custome stopwords 58.47


##### Stemming and Lemmatization

In [55]:
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.SnowballStemmer('english')

for word in ['studies', 'studied', 'studying', 'loves', 'loved']:
  print(word, lemmatizer.lemmatize(word), stemmer.stem(word))

studies study studi
studied studied studi
studying studying studi
loves love love
loved loved love


### Removing Emojis

In [20]:
data_no_animate_emoji = convert_animated_emojis(data)
data_no_emoji = convert_text_emoji(data_no_animate_emoji)

In [8]:
model = LogisticRegression(solver = 'liblinear', max_iter= 1000, random_state=0)

In [59]:
run_model(data_no_emoji, data_labels, model=model)

0.633

### Compressing

In [37]:
compresed_data = compress(data_no_emoji)

In [53]:
run_model(compresed_data, data_labels, model=model)

0.634

In [12]:
show_emoji('xf0x9fx98x82')

'😂'

### Visualize and find best accuracy for multiple inputs

In [ ]:
X = range(1000, 4001, 200)
Y = []
best_acc = 0
for n in X:
  data = list(set(most_common(pos_lem_stem + neg_lem_stem, n)))
  acc = run_model(data)
  if acc > best_acc:
    best_acc = acc
  Y.append(acc) 

print('Best Accuracy: ', best_acc)
plt.plot(X, Y)
plt.show()

### Lematization and Stemming

In [ ]:
class LemmaTokenizer(object):
  def __init__(self):
    self.wnl = WordNetLemmatizer()
  def __call__(self, articles):
    #a = [self.wnl.lemmatize(compress(t)) for t in word_tokenize(articles) if '/' not in t and len(t) >= 2 and t.isalpha()]
    #a = [self.wnl.lemmatize(compress(t), map_pos(nltk.pos_tag(t)[0][1])) for t in word_tokenize(articles) if '/' not in t and t.isalpha()]
    a = [self.wnl.lemmatize(t, map_pos(nltk.pos_tag(t)[0][1])) for t in word_tokenize(articles) if '/' not in t and t.isalpha()]
    return [i for i in a if len(i) > 1]

lemmatizer = WordNetLemmatizer()
stemmer = nltk.stem.SnowballStemmer('english')

### Cleaning Data

In [ ]:
posStr = pos.lower() 
negStr = neg.lower()

posStr2 = posStr.replace("\'", '')
negStr2 = negStr.replace("\'", '')

posStr = pos.lower().replace("n\'t", ' not')
negStr = neg.lower().replace("n\'t", ' not')

pos_cleaned = re.findall("[a-z][a-z]+", posStr2)
neg_cleaned = re.findall("[a-z][a-z]+", negStr2)

pos_lem_stem = [ lemmatizer.lemmatize(i) for i in pos_cleaned ]
neg_lem_stem = [ lemmatizer.lemmatize(i) for i in neg_cleaned ]

### Finding Misclassified Inputs

In [ ]:
misclassified_samples_pos = X_val[(y_val != y_pred) & (y_pred == 'pos')]
misclassified_samples_neg = X_val[(y_val != y_pred) & (y_pred == 'neg')]
res = vectorizer.inverse_transform(misclassified_samples_neg)

In [10]:
#model = LogisticRegression(**model_params, solver = 'liblinear', max_iter= 1000, random_state=0)
model = LogisticRegression(solver = 'liblinear', max_iter= 1000, random_state=0)

# for i in range(1000, 10001, 500):
#   vocab = [i[0] for i in Counter(tokens).most_common(i)]
#   run_model(data2_string_per_line, {'vocabulary':vocab})  # best 73.1

# black_list = ['xe2x80xa6','xe2x80x9c', 'xe2x80x9d', 'xefxb8x8f', 'xe2x80x99']

# for i in range(1000, 7501, 100):
#   vocab = [i[0] for i in Counter(tokens).most_common(i)]
#   print(i, run_model(data2_string_per_line, {'vocabulary':vocab}, model=model))  # best 73.1

0.635
